# Ejercicio FIFA

## Diestros vs Zurdos

¿Te acuerdas del famoso problema de los diestros y los zurdos? Como vimos, con un algoritmo lineal no era posible predecir demasiado bien si un jugador era zurdo o diestro. Con el modelo basado en árboles de decisión, mejorábamos nuestro porcentaje de acierto (con los datos equilibrados).

1. Implementa el caso con un modelo KNN. ¿Conseguirás un mejor score (con datos balanceados) que con el modelo basado en árboles de decisión? Puedes crear todas las variables que quieras, al igual que hemos hecho para los otros modelos:

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [3]:
df = pd.read_csv("../../../data/FIFA20.csv")

df['año'] = df['dob'].apply(lambda x: int(x[0:4]))
df['mes'] = df['dob'].apply(lambda x: int(x[5:7]))
df['dia'] = df['dob'].apply(lambda x: int(x[8:10]))

df = df.dropna(subset=["team_position"])

df['money_pos'] = df.groupby("team_position").apply(lambda x: x['value_eur']/x['value_eur'].mean()).values

def position(x):
    x = str(x)
    if x[0] == 'R' and x not in ['RES']:
        return 'R'
    elif x[0] == 'L':
        return 'L'
    else:
        return ''

df['team_position'] = df['team_position'].apply(lambda x: position(x))

feature_cols = ['overall', 'potential', 'value_eur', 'año', 'mes', 'money_pos', 'team_position', 'dia']
pred_col = 'preferred_foot'

X = pd.get_dummies(df[feature_cols])
Y = df[pred_col]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, random_state=42)

knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train, Y_train)

print(f"Train: {knn.score(X_train, Y_train)}")
print(f"Test: {knn.score(X_test, Y_test)}")

Train: 0.768176778076984
Test: 0.7544345898004434


In [4]:
best = 0
best_i=0

for k in range(1, 51, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    new_score = knn.score(X_test, Y_test)
    
    if new_score > best:
        best = new_score
        best_i = k
    

In [5]:
len(df)

18038

In [6]:
best

0.7682926829268293

Parece bueno, pero tenemos unos datos desbalanceados, así que veamos cómo funciona de verdad:

In [364]:
df = pd.read_csv("../../../data/FIFA20.csv")

df['año'] = df['dob'].apply(lambda x: int(x[0:4]))
df['mes'] = df['dob'].apply(lambda x: int(x[5:7]))
df['dia'] = df['dob'].apply(lambda x: int(x[8:10]))

df = df.dropna(subset=["team_position"])

# En clase hemos hecho la separación de otra forma, sin embargo, también podemos utilizar el método 'sample(n)' para seleccionar registros aleatorios:
df_zurdos = df[df['preferred_foot'] == 'Left']
df_diestros = df[df['preferred_foot'] == 'Right'].sample(len(df_zurdos))

# Lo ordenamos por index para que no estén ordenados por pierna, ya que originalmente estaban desordenados:
df = df_zurdos.append(df_diestros).sort_index()

df['money_pos'] = df.groupby("team_position").apply(lambda x: x['value_eur']/x['value_eur'].mean()).values
df['money_ovr'] = df.groupby("overall").apply(lambda x: x['value_eur']/x['value_eur'].mean()).values
df['money_pot'] = df.groupby("potential").apply(lambda x: x['value_eur']/x['value_eur'].mean()).values
df['wage_pos'] = df.groupby("team_position").apply(lambda x: x['wage_eur']/x['wage_eur'].mean()).values
df['wage_ovr'] = df.groupby("overall").apply(lambda x: x['wage_eur']/x['wage_eur'].mean()).values
df['wage_pot'] = df.groupby("potential").apply(lambda x: x['wage_eur']/x['wage_eur'].mean()).values

df = df.fillna(0)

def position(x):
    x = str(x)
    if x[0] == 'R' and x not in ['RES']:
        return 'R'
    elif x[0] == 'L':
        return 'L'
    else:
        return ''

df['team_position2'] = df['team_position'].apply(lambda x: position(x))

feature_cols = ['team_position', 'money_pos']
pred_col = 'preferred_foot'

X = pd.get_dummies(df[feature_cols])
Y = df[pred_col]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.3, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, Y_train)

print(f"Train: {knn.score(X_train, Y_train)}")
print(f"Test: {knn.score(X_test, Y_test)}")

Train: 0.709639564124057
Test: 0.5733281188893234
